# House Price Predictions - Modelling

In this notebook we will explore and make predictions on the housing dataset from Kaggle:\
https://www.kaggle.com/c/house-prices-advanced-regression-techniques/


* Setup
    * Import libraries
    * Read the data
* Linear Models
* Ensemble Methods

# Setup
#### Import libraries

In [46]:
# data science
import pandas as pd
import numpy as np

# warnings
import warnings
def ignore_warn(*args, **kwargs): pass
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.warn = ignore_warn


# config
pd.options.display.max_columns = None

# modelling
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LinearRegression, ElasticNet, Ridge, Lasso, Lars, LassoLars, OrthogonalMatchingPursuit, BayesianRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb

##### Read the Data

In [18]:
# data frames
df_train = pd.read_csv('files/train_processed.csv')
df_test = pd.read_csv('files/test_processed.csv')
df_all = pd.concat([df_train, df_test])

# keep track
ntrain = df_train.shape[0]
ntest = df_test.shape[0]

# print shape
print('rows/cols:', df_train.shape[0], df_train.shape[1], 'train')
print('rows/cols:', df_test.shape[0], df_test.shape[1], 'test')
print('rows/cols:', df_all.shape[0], df_all.shape[1], 'train+test')

rows/cols: 1458 221 train
rows/cols: 1459 220 test
rows/cols: 2917 221 train+test


In [19]:
# data frames
y_train = df_train['SalePrice'].copy()
x_train = df_train.drop(columns='SalePrice')
x_test = df_test

#### Validation Function

In [20]:
n_folds = 10

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(x_train.values)
    rmse= np.sqrt(-cross_val_score(model, x_train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

# Base Models

* https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard
* https://dataaspirant.com/ensemble-methods-bagging-vs-boosting-difference/
* https://www.math.unipd.it/~aiolli/corsi/1213/aa/user_guide-0.12-git.pdf

#### Base Models

In [47]:
pipelines = {
    'OLS' : make_pipeline(RobustScaler(), LinearRegression()),
    'Lasso' : make_pipeline(RobustScaler(), Lasso(alpha =0.0004, random_state=1)),
    'Ridge' : make_pipeline(RobustScaler(), Ridge()),
    'ENet' : make_pipeline(RobustScaler(), ElasticNet(alpha=0.0004, l1_ratio=.9, random_state=3)),
    'OMP' : make_pipeline(RobustScaler(), OrthogonalMatchingPursuit()),
    'BRidge' : BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
                fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
                normalize=False, tol=0.001, verbose=False)
}

for p in pipelines:
    score = rmsle_cv(pipelines[p])
    print(p, "\t score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

OLS 	 score: 0.1181 (0.0163)
Lasso 	 score: 0.1087 (0.0145)
Ridge 	 score: 0.1130 (0.0156)
ENet 	 score: 0.1087 (0.0145)
OMP 	 score: 0.1209 (0.0149)
BRidge 	 score: 0.1103 (0.0140)


#### Gradient Boosting Models

In [ ]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1,
                             verbosity=0)

model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11,
                             verbosity=-1)

In [ ]:
#score1 = rmsle_cv(GBoost)
#score2 = rmsle_cv(model_xgb)
#score3 = rmsle_cv(model_lgb)

In [ ]:
print("GBoost score: {:.4f} ({:.4f})".format(score1.mean(), score1.std()))
print("XGBoost score: {:.4f} ({:.4f})".format(score2.mean(), score2.std()))
print("LGBoost score: {:.4f} ({:.4f})".format(score3.mean(), score3.std()))

#### Inspect Predictions

In [ ]:
models = {
    'Lasso': lasso,
    'ENet': ENet,
    'KRR': KRR,
    'GBoost': GBoost,
    'model_xgb': model_xgb,
    'model_lgb': model_lgb,
}

In [ ]:
# SalePrice
df = np.expm1(y_train).to_frame()

# Prediction Models
for m in models:
    mfit = models[m].fit(x_train.values, y_train)
    pred = mfit.predict(x_train.values)
    df[m] = np.expm1(pred)

In [ ]:
df

#### Stacking Models - Averaged

In [ ]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)   

In [ ]:
averaged_models = AveragingModels(models = (ENet, GBoost, KRR, lasso))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

#### Stacking Models - Meta model

In [ ]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [ ]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet, GBoost, KRR),
                                                 meta_model = lasso)

score = rmsle_cv(stacked_averaged_models)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

#### Stacked Regressor

In [ ]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
stacked_averaged_models.fit(x_train.values, y_train)
stacked_train_pred = stacked_averaged_models.predict(x_train.values)
#stacked_pred = np.expm1(stacked_averaged_models.predict(test.values))
print(rmsle(y_train, stacked_train_pred))

In [ ]:
model_xgb.fit(x_train, y_train)
xgb_train_pred = model_xgb.predict(x_train)
#xgb_pred = np.expm1(model_xgb.predict(test))
print(rmsle(y_train, xgb_train_pred))

In [ ]:
model_lgb.fit(x_train, y_train)
lgb_train_pred = model_lgb.predict(x_train)
#lgb_pred = np.expm1(model_lgb.predict(test.values))
print(rmsle(y_train, lgb_train_pred))

In [ ]:
'''RMSE on the entire Train data when averaging'''

print('RMSLE score on train data:')
print(rmsle(y_train,stacked_train_pred*0.70 +
               xgb_train_pred*0.15 + lgb_train_pred*0.15 ))

#### Ensemble Prediction

In [ ]:
ensemble = stacked_pred*0.70 + xgb_pred*0.15 + lgb_pred*0.15